In [158]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

############ load and digitize the data
data=pd.read_csv('/home/ebotian/MCM/tennis.csv')
data = pd.get_dummies(data, columns=['winner_shot_type','serve_width','serve_depth','return_depth'])
############

#feature_list=data.columns.tolist()

#print(feature_list)

############# pre-process the data
# fill nan with 0, and replace AD with 50
data = data.fillna(0)
data = data.replace('AD', 50.0)
data['point_victor']=data["point_victor"].replace(2,0)
print(data.iloc[:,15])

# split the data into different match
grouped = dict(tuple(data.groupby(data['match_id'].ne(data['match_id'].shift()).cumsum())))

# Rename the subdata
subdata = {df['match_id'].iloc[0]: df for _, df in grouped.items()}

# Create a new dataset from the first column, excluding duplicates
match_id = pd.DataFrame(data.iloc[:, 0].drop_duplicates()).iloc[:,0].tolist()
print(match_id[0])
##############

# check all data
#data.to_csv('output.csv',index=False)

0       0
1       1
2       0
3       1
4       1
       ..
7279    1
7280    1
7281    0
7282    1
7283    1
Name: point_victor, Length: 7284, dtype: int64
2023-wimbledon-1301


In [159]:
# Convert the timestamp column to datetime format
subdata[match_id[0]]['elapsed_time'] = pd.to_timedelta(subdata[match_id[0]]['elapsed_time'])

# Calculate the time difference between consecutive rows
subdata[match_id[0]]['time_diff'] = subdata[match_id[0]]['elapsed_time'].diff()

# Calculate the 5th and 95th percentiles
lower_threshold = subdata[match_id[0]]['time_diff'].quantile(0.05)
upper_threshold = subdata[match_id[0]]['time_diff'].quantile(0.95)

# Exclude the top 5% and bottom 5% of periods
filtered_diff = subdata[match_id[0]]['time_diff'][(subdata[match_id[0]]['time_diff'] > lower_threshold) & (subdata[match_id[0]]['time_diff'] < upper_threshold)]

# Calculate the average of the remaining intervals
average_interval = filtered_diff.mean()

print(f'Average interval (excluding top 5% and bottom 5%): {average_interval}')

# Convert the time differences to integer seconds
subdata[match_id[0]]['time_diff'] = subdata[match_id[0]]['time_diff'].dt.total_seconds()

# Replace 'NaT' values with 0
subdata[match_id[0]]['time_diff'] = subdata[match_id[0]]['time_diff'].fillna(0).astype(int)
#fill nan with 0
#subdata[match_id[0]]['time_diff'] = subdata[match_id[0]]['time_diff'].fillna(0)

Average interval (excluding top 5% and bottom 5%): 0 days 00:00:43.784090909


In [160]:
############ add features

add_feature=["score_diff"]
#add_feature_2='p1_score_ratio'
#add_feature_3='p2_score_ratio'
#add_feature_4='p1_break_pt_efficiency'
#add_feature_5='p2_break_pt_efficiency'

In [161]:
########## defining the new features

#example: subdata[match_id[0]][add_feature[0]] = subdata[match_id[0]]['p1_games'] - subdata[match_id[0]]['p2_games']


#data[add_feature_1] = data['p1_games'] - data['p2_games']
#data[add_feature_2] = data['p1_winner'] / (data['p1_winner'] + data['p2_winner'])
#data[add_feature_3] = data['p2_winner'] / (data['p1_winner'] + data['p2_winner'])
#data[add_feature_4] = data['p1_break_pt_won'] / (data['p1_break_pt_won'] + data['p1_break_pt_missed'])
#data[add_feature_5] = data['p2_break_pt_won'] / (data['p2_break_pt_won'] + data['p2_break_pt_missed'])


In [162]:
# split the data into target and features
target=pd.DataFrame(subdata[match_id[0]]["point_victor"])

subdata[match_id[0]]=subdata[match_id[0]].drop(columns=["point_victor"])
features=subdata[match_id[0]].iloc[:,4:]
#print(features)

In [163]:
#print(features.isna().sum())

In [164]:

# 数据划分
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)

#print(X_train.isna().sum())

In [166]:

# 数据标准化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 模型训练
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train_scaled, y_train)

#print(X_test_scaled)

/usr/lib/python3.11/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(random_state=42)

In [167]:

# 模型预测
y_pred = clf.predict(X_test_scaled)

print(y_pred)

[1 0 0 1 0 0 1 0 0 0 0 1 0 0 1 0 1 1 0 1 0 0 0 1 1 1 1 1 1 1 0 0 1 0 0 1 0
 1 1 0 1 0 0 1 0 0 1 1 1 0 1 0 0 0 1 1 1 0 1 0 1 0 1 1 1 0 1 0 0 1 0 1 0 0
 1 1 1 1 0 0 0 1 1 0 1 1 1 0 0 0]


In [168]:

# 模型评估
accuracy = accuracy_score(y_test, y_pred)
print(f'模型准确率: {accuracy}')

# 可以考虑输出特征重要性，帮助理解哪些因素对预测结果影响较大
importances = clf.feature_importances_
feature_names = X_train.columns
feature_importances = pd.Series(importances, index=feature_names).sort_values(ascending=False)
print(feature_importances)

模型准确率: 0.9
p2_unf_err            0.091219
p1_unf_err            0.088234
server                0.079603
p2_winner             0.063790
p2_distance_run       0.060896
rally_count           0.054003
p1_distance_run       0.053502
p1_winner             0.046627
speed_mph             0.039556
p2_net_pt_won         0.036565
time_diff             0.035225
p1_points_won         0.034677
point_no              0.026210
p2_points_won         0.025795
p1_score              0.022302
game_victor           0.020121
game_no               0.018135
p2_games              0.016764
p1_net_pt_won         0.015626
p2_score              0.015496
p1_games              0.014893
serve_no              0.011759
p2_ace                0.009918
serve_width_BW        0.008863
return_depth_ND       0.008705
return_depth_D        0.007917
set_no                0.007223
p2_net_pt             0.006832
p1_double_fault       0.006710
serve_width_BC        0.005751
serve_depth_CTL       0.005520
p1_sets               0.0054